In [1]:
import os
os.environ['PYTHONPATH'] = '/Users/g.sarapulov/Projects/draft/archtorch'
import sys
sys.path.append('/Users/g.sarapulov/Projects/draft/archtorch')

In [2]:
from data.input import FileParser, DataSampler
from loss.margin import ContrastiveLoss
from network.siamese import Siamese, Encoder
from train.fit import NetworkTrainer

In [3]:
train_path = 'train_db/'
test_path = 'test_db/'
parser = FileParser('.txt', ' ')
train_dataset = DataSampler(parser, train_path)
test_dataset = DataSampler(parser, test_path)

In [4]:
model = Siamese(Encoder())
loss = ContrastiveLoss(margin=1.)
trainer = NetworkTrainer(train_dataset, test_dataset, log_interval=1000)

In [5]:
model = trainer.fit_model(model, loss, lr=1e-3, n_epochs=5)

Train: [0/52284 (0%)]	Loss: 0.191492
Epoch: 1/5. Train set: Average loss: 0.1422
Epoch: 1/5. Validation set: Average loss: 0.1769
Train: [0/52284 (0%)]	Loss: 0.146064
Epoch: 2/5. Train set: Average loss: 0.1379
Epoch: 2/5. Validation set: Average loss: 0.1730
Train: [0/52284 (0%)]	Loss: 0.142462
Epoch: 3/5. Train set: Average loss: 0.1374
Epoch: 3/5. Validation set: Average loss: 0.1847
Train: [0/52284 (0%)]	Loss: 0.142973
Epoch: 4/5. Train set: Average loss: 0.1371
Epoch: 4/5. Validation set: Average loss: 0.1719
Train: [0/52284 (0%)]	Loss: 0.141518
Epoch: 5/5. Train set: Average loss: 0.1376
Epoch: 5/5. Validation set: Average loss: 0.1767
